## 第2週 3日目

### 概要
2日目の内容に以下を追加する。

1. 異なるモデル
1. 構造化出力
1. ガードレール

また、余談だが、

- OpenAI Agents SDKでは、色々なLLMを使用できるがFunctionCallingは専用の学習がされているLLMである必要があるので、サイズの小さなローカルLLMでは適切に動作しない可能性がある。
- Anthropic の Claude は、 OpenAI 互換 API ではないが、Open Router などを使用することで、OpenAI Agents SDK でも使用することができるようになるとのこと。
- ガードレール はエージェントの入力の「一番最初」と「一番最後」にしか設定できない。このフレームワークでは、アチコチにガードレールを設定することはできない。

## 準備

In [1]:
import os
from typing import Dict
from pydantic import BaseModel

from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput

import resend # import sendgrid
#from sendgrid.helpers.mail import Mail, Email, To, Content

In [2]:
load_dotenv(override=True)

True

In [3]:
# キープレフィックスを印刷して、デバッグに役立てます

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI APIキーが存在し、{openai_api_key[:8]}で始まります。")
else:
    print("OpenAI APIキーが設定されていません")
    
if anthropic_api_key:
    print(f"Anthropic APIキーが存在し、{anthropic_api_key[:7]}で始まります。")
else:
    print("Anthropic APIキーは設定されていません（これはオプションです）")

if google_api_key:
    print(f"Google APIキーが存在し、{google_api_key[:2]}で始まります。")
else:
    print("Google APIキーが設定されていません（これはオプションです）")

if deepseek_api_key:
    print(f"DeepSeek APIキーが存在し、{deepseek_api_key[:3]}で始まります。")
else:
    print("DeepSeek APIキーは設定されていません（これはオプションです）")

if groq_api_key:
    print(f"Groq APIキーが存在し、{groq_api_key[:4]}で始まります。")
else:
    print("Groq APIキーは設定されていません（これはオプションです）")

OpenAI APIキーが存在し、sk-proj-で始まります。
Anthropic APIキーは設定されていません（これはオプションです）
Google APIキーが存在し、AIで始まります。
DeepSeek APIキーは設定されていません（これはオプションです）
Groq APIキーは設定されていません（これはオプションです）


### 変更が無い部分（１）

In [4]:
#instruction1「あなたはComplAIで営業担当者として働いています。
# ComplAIは、AIを活用したSOC2コンプライアンスの確保と監査対策のためのSaaSツールを提供する企業です。
# あなたは「プロフェッショナルで真摯な」コールドメールを作成しています。」
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

# instructions2「あなたはComplAIで営業担当者として働いています。
# ComplAIは、AIを活用したSOC2コンプライアンスの確保と監査対策のためのSaaSツールを提供する企業です。
# あなたは「返信を期待できる、機知に富んだ魅力的な」コールドメールを作成しています。」
instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

# instructions3「あなたはComplAIで営業担当者として働いています。
# ComplAIは、AIを活用したSOC2コンプライアンスの確保と監査対策のためのSaaSツールを提供する企業です。
# あなたは「簡潔で要点を押さえた」コールドメールを作成しています。」
instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
# コールド・セールス・メールの件名を書くことができます。メッセージの内容に応じて、返信を期待できる件名を書く必要があります。
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

# テキストメールの本文を HTML メールの本文に変換。マークダウンが含まれている可能性のあるテキストメールの本文を、シンプルで明確、かつ魅力的なレイアウトとデザインの HTML メールの本文に変換する。
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

# 件名考案エージェントをツール化
subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

# MD → HTML変換エージェントをツール化
html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [6]:
# テキストメールからHTMLに改修されている。
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    
    #sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    resend.api_key=os.getenv("RESEND_API_KEY")
    
    #from_email = Email("ed@edwarddonner.com")  # 検証済みの送信者に変更します
    #to_email = To("ed.donner@gmail.com")  # 受信者に変更します
    #content = Content("text/html", html_body)
    #mail = Mail(from_email, to_email, subject, content).get()
    #sg.client.mail.send.post(request_body=mail)
    
    response = resend.Emails.send({
        "from": "onboarding@resend.dev",
        "to": "nishi_74322014@ksj.biglobe.ne.jp",
        "subject": subject,
        "html": html_body
    })

    return {"status": "success"}

In [7]:
tools = [subject_tool, html_tool, send_html_email]

In [8]:
# あなたはメールのフォーマッター兼送信者です。送信するメールの本文を受け取ります。
# まず、subject_writer ツールを使用してメールの件名を作成し、次に html_converter ツールを使用して本文を HTML に変換します。
# 最後に、send_html_email ツールを使用して、件名と HTML 本文を含むメールを送信します。
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

# メール関係のハンドリングを（メールをHTMLに変換して送信）するエージェント
emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

### 異なるモデルの使用
OpenAI互換のエンドポイントを備えた異なるモデルを使用するエージェント（個人的に利用可能なラインナップに変更）

In [9]:
GEMINI_BASE_URL    = "https://generativelanguage.googleapis.com/v1beta/openai/"
#DEEPSEEK_BASE_URL  = "https://api.deepseek.com/v1"
#GROQ_BASE_URL      = "https://api.groq.com/openai/v1"

In [10]:
gemini_client   = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
#deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
#groq_client     = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

In [11]:
gemini_model    = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
#deepseek_model  = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
#llama3_3_model  = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [12]:
sales_agent1 = Agent(name="gpt-4.1 Professional Sales Agent", instructions=instructions1, model="gpt-4.1")
sales_agent2 = Agent(name="gpt-5 Engaging Sales Agent", instructions=instructions2, model="gpt-5")
sales_agent3 = Agent(name="Gemini Busy Sales Agent", instructions=instructions3, model=gemini_model)
#sales_agentX = Agent(name="DeepSeek Sales Agent", instructions=instructionsX, model=deepseek_model)
#sales_agentY = Agent(name="Llama3.3 Sales Agent",instructions=instructionsY,model=llama3_3_model)

In [13]:
# コールド・セールス・メールを書く
description = "Write a cold sales email"

# エージェントをツールに変換
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

### 変更が無い部分（２）

In [14]:
# （コールド・セールス・メール作成エージェントを）ツールにまとめる。
tools = [tool1, tool2, tool3]
print(tools)

# （メール関係のハンドリングのエージェントに）ハンドオフする。
handoffs = [emailer_agent]
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7ff2edc28180>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7ff2edc28540>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'requi

In [15]:
# メール送信の所がツール使用からハンドオフに変更されている。

# 学生Guillermo Fのおかげで指示の改善

# あなたはComplAIの営業マネージャーです。あなたの目標は、sales_agentツールを使って、最適なコールド・セールス・メールを1通見つけることです。
# 
# 以下の手順を慎重に実行してください。
# 1. 下書きの作成：3つのsales_agentツールすべてを使って、3種類のメール下書きを作成します。3つの下書きがすべて完成するまで、先に進めないでください。
# 2. 評価と選択：下書きを確認し、最も効果的だと判断した上で、最適なメールを1通選びます。最初の結果に満足できない場合は、ツールを複数回使用できます。
# 3. 送信のためのハンドオフ：採用されたメールの下書きのみをEmail Managerエージェントに渡します。Email Managerがフォーマットと送信を行います。
# 
# 重要なルール：
# - 下書きの作成には、sales_agentツールを使用する必要があります。自分で作成しないでください。
# - Email Managerに引き継ぐメールは1通のみで、複数回は引き継がないでください。

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

# Sales Manager
sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    # （エージェントを）ツールにまとめる。
    tools=tools,
    # （メール関係のハンドリングは）ハンドオフする。
    handoffs=handoffs,
    model="gpt-4o-mini")

# アリスから「CEO様」宛て「コールド・セールス・メール」を送信
message = "Send out a cold sales email addressed to Dear CEO from Alice"

# OpenAI API PF → Dashboard → Logs → Traces の 
with trace("Automated SDR"):
    # アリスから「CEO様」宛て「コールド・セールス・メール」を送信
    result = await Runner.run(sales_manager, message)

### トレースをチェックしてください：

https://platform.openai.com/traces

## 構造化出力とガードレール

In [16]:
# 構造化出力するエージェント
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

# ユーザーがあなたに実行して欲しい操作の中に、誰かの個人名が含まれているかどうかを確認します。
guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [17]:
# 入力ガードレールの設定
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    # 入力ガードレールの中で構造化出力するエージェントを動かし、
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    # 出力の is_name_in_message (bool) を確認し、tripwire_triggeredで、違反の有無を通知
    is_name_in_message = result.final_output.is_name_in_message # 構造化出力のプロパティ
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [18]:
# 構造化出力＋ガードレール＝慎重な営業マネージャー
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

## トレースをチェックしてください：

https://platform.openai.com/traces

In [19]:
# ガードレールで止まる

# アリスから「CEO様」宛て「コールド・セールス・メール」を送信
message = "Send out a cold sales email addressed to Dear CEO from Alice"

# OpenAI API PF → Dashboard → Logs → Traces の 
with trace("Protected Automated SDR"):
    # アリスから「CEO様」宛て「コールド・セールス・メール」を「慎重に」送信
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [20]:
# ガードレールを通過

# 事業開発責任者から「CEO様」宛て「コールド・セールス・メール」を送信
message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

# OpenAI API PF → Dashboard → Logs → Traces の 
with trace("Protected Automated SDR"):
    # 事業開発責任者から「CEO様」宛て「コールド・セールス・メール」を「慎重に」送信
    result = await Runner.run(careful_sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">演習</h2>
            <span style="color:#ff7800;">
                • さまざまなモデルを試す<br/>
                • 入出力のガードレールを追加する<br/>
                • メール生成に構造化された出力を使用する
            </span>
        </td>
    </tr>
</table>